In [1]:
import unittest
import sqlite3
import requests
from bs4 import BeautifulSoup 
from mock import MagicMock
import time
from urllib.parse import urljoin

In [68]:
class spyder:
    def __init__( self ,links,base_url,depth ):
        self.base_url = base_url
        target_links={}
        for i in links:
            target_links[i]=0 
        self.target_links = target_links
        self.depth = depth
    
    def get_crawler(self):
        self.result_crawler = self.crawl(self.base_url,self.depth,0,set())
        return self.result_crawler
    
    def get_check_domain(self):
        self.check_domain_result = self.check_domain(self.base_url,self.get_crawler())
        return self.check_domain_result
    
    def get_check_not_domain(self):
        self.check_not_domain_result = self.check_not_domain(self.base_url,self.get_crawler())   
        return self.check_not_domain_result
    
    def get_check_ref(self):
        self.check_ref_result = self.check_ref(self.get_check_not_domain(),self.target_links)
        return self.check_ref_result
    
    def crawl(self,url,n, depth,visited):
        if depth < n :
            visited.add(url)
            headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
            time.sleep(0.3)
            response = requests.get(url,headers=headers)
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                soup = BeautifulSoup(response.text, 'lxml')
            links = soup.find_all('a')
            links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
            links = [urljoin(url, link) for link in links if link]

            for link in links:
                if link not in visited:
                    link = link.replace(' ','')
                    visited.add(link)
                    if link.startswith(url):
                        self.crawl(link,n=n,depth=depth+1, visited=visited)
        return visited
    
    def check_domain(self,base_url,links):
        result= set()
        for link in links :
            if link.startswith(base_url):
                result.add(link)
        return result
    
    def check_not_domain(self,base_url,links):
        result= set()
        for link in links :
            if not link.startswith(base_url):
                result.add(link)
        return result
    
    def check_ref(self,links,target_links):
        for i in links:
            for j in target_links:
                if i.startswith(j):
                    target_links[j]+=1
        return target_links

In [102]:
class TestSpyder(unittest.TestCase):
    def setUp(self):
        self.base_url = "https://www.bbc.com/news"
        self.links = ["https://www.bbc.com"]
        self.depth = 1
        self.spyder = spyder(self.links, self.base_url, self.depth)
        
    def test_crawl(self):
        visited = self.spyder.crawl(self.base_url, self.depth, 0, set())
        self.assertIn(self.base_url, visited)
            
    def test_check_domain(self):
        links = self.links
        domain_links = self.spyder.check_domain("https://www.bbc.com", ["https://www.bbc.com/news"])
        self.assertIn("https://www.bbc.com/news", domain_links)
        self.assertNotIn("https://www.bbc.co.uk", domain_links)
        
    def test_check_not_domain(self):
        links = self.links
        not_domain_links = self.spyder.check_not_domain(self.base_url, links)
        self.assertNotIn("https://www.bbc.com/news", not_domain_links)
        self.assertIn("https://www.bbc.com", not_domain_links)
        
    def test_check_ref(self):
        links = ["https://www.example.com/about"]
        target_links = {"https://www.example.com": 0}
        ref_counts = self.spyder.check_ref(links, target_links)
        self.assertEqual(ref_counts,{"https://www.example.com": 1})
        

In [103]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestSpyder)

In [104]:
unittest.TextTestRunner().run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.645s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>